In [1]:
import os, sys, re, json, time, unittest
import itertools, collections
import re
import operator
from sklearn import datasets, linear_model
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [143]:
import nltk
#read in data to clean and separate into training and test set
good = []
bad = []
score = []
pre_tags = []
tags = []
pre_lines = []
a = 0
b = 0
#1 is good
#0 is not
for line in open('listings_scores.csv').readlines():
    data = line.split('\t')
    clean_score = re.sub("[^0-9]", "", data[1])
    desc = data[0]
    if clean_score != '' and int(clean_score) <= 100 and int(clean_score) > 0:
        if int(clean_score) >= 90:
            for each in desc.lower().split('.'):
                if each != ' ':
                    good.append(re.sub('[^A-Za-z]', ' ', each))
                    pre_lines.append((re.sub('[^A-Za-z]', ' ', each), 'good'))
                    score.append(int(clean_score))
                    tags.append(int(1))
                    pre_tags.append(int(1))
                    a += 1
        elif int(clean_score) < 90:
            for each in desc.lower().split('.'):
                if each != ' ':
                    pre_lines.append((re.sub('[^A-Za-z]', ' ', each), 'bad'))
                    bad.append(re.sub('[^A-Za-z]', ' ', each))
                    score.append(int(clean_score))
                    pre_tags.append(int(0))
                    b += 1

print a
print b

189616
22763


In [146]:
from sklearn.utils import resample

# Separate majority and minority classes
lines_majority = good
lines_minority = bad
 
# Upsample minority class
bad_upsampled = resample(lines_minority, 
                         replace=True,     # sample with replacement
                         n_samples=189616,    # to match majority class
                         random_state=123) # reproducible results
 
    
# Combine majority class with upsampled minority class
print len(bad_upsampled)
print len(good)


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from random import shuffle


new_list = []

for each in good:
    new_list.append((each, int(1)))

print len(new_list)
bad_upsampled = list(bad_upsampled)
for each in bad_upsampled:
    new_list.append((each, int(0)))

shuffle(new_list)

new_lines = []
new_tags = []

for each in new_list:
    new_lines.append(each[0])
    new_tags.append(each[1])
                     
                     
print len(new_lines)
print len(new_tags)


189616
189616
189616
379232
379232


In [190]:
def word_embedding(descriptions):
    words = []
    word_count = 0
    for word in descriptions.split(" "):
        words.append(word)
    features = []
    for each in words:
        word_count += 1
    
    features.append(word_count)
    return features


featuresets2 = [word_embedding(descriptions) for (descriptions, tag) in new_list]
print len(featuresets2)
print max(featuresets2)
#print new_list[0]
#featuresets = [(word_embedding(descriptions), tag) for (descriptions, tag) in new_list]

379232
[542]


In [223]:
count = 0
total = 0
under_10 = 0

under_20 = 0

under_30 = 0

under_40 = 0

under_50 = 0

under_60 = 0

under_70 = 0

under_80 = 0

under_90 = 0

under_100 = 0

over_100 = 0


for each in featuresets2:
    if each[0] <= 10:
        under_10 += 1
    elif each[0] <=20 and each[0] > 10:
        under_20 += 1
    elif each[0] <=30 and each[0] > 20:
        under_30 += 1
    elif each[0] <=40 and each[0] > 30:
        under_40 += 1
    elif each[0] <=50 and each[0] > 40:
        under_50 += 1
    elif each[0] <=60 and each[0] > 50:
        under_60 += 1
    elif each[0] <=70 and each[0] > 60:
        under_70 += 1
    elif each[0] <=80 and each[0] > 70:
        under_80 += 1
    elif each[0] <=90 and each[0] > 80:
        under_90 += 1
    elif each[0] <=100 and each[0] > 90:
        under_100 += 1
    elif each[0] > 100:
        over_100 += 1
         
        
        
        
        
    total += each[0]
    count +=1
#print total
#print count

print under_10 

print under_20 

print under_30 

print under_40 

print under_50 

print under_60 

print under_70 

print under_80 

print under_90

print under_100

print over_100


#print total/count
#print len(featuresets)

97858
143206
77439
31200
13664
6688
3328
1755
1131
770
2193


In [ ]:
sentences = []
feature_tags = []

for each in featuresets:
    sentences.append(each[0])
    if each[1] == 1:
        feature_tags.append(1)
    elif each[1] == 0:
        feature_tags.append(0)
X_train = sentences[:len(sentences)/2]
y_train = feature_tags[:len(feature_tags)/2]
X_test = sentences[len(sentences)/2:]
y_test = feature_tags[len(feature_tags)/2:]

In [ ]:
max_review_length = 25
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

In [237]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_pad_train, y_train, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, 20, 32)            718976    
_________________________________________________________________
lstm_44 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
189616/189616 [==============================] - 83s 439us/step - loss: 0.6638 - acc: 0.5910
Epoch 2/5
189616/189616 [==============================] - 86s 452us/step - loss: 0.6354 - acc: 0.6316
Epoch 3/5
189616/189616 [==============================] - 85s 449us/step - loss: 0.6175 - acc: 0.6491
Epoch 4/5
189616/189616 [==============================] - 85s 451us/step - loss: 0.60

In [ ]:
max_review_length = 25
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

In [239]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_pad_train, y_train, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_42 (Embedding)     (None, 25, 32)            718976    
_________________________________________________________________
lstm_45 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
189616/189616 [==============================] - 103s 543us/step - loss: 0.6647 - acc: 0.5864
Epoch 2/5
189616/189616 [==============================] - 97s 509us/step - loss: 0.6377 - acc: 0.62961s - loss: 0.
Epoch 3/5
189616/189616 [==============================] - 95s 503us/step - loss: 0.6190 - acc: 0.6473
Epoch 4/5
189616/189616 [==============================] - 97s 514us/ste

In [258]:
max_review_length = 20
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

20
1


In [260]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[recall, precision,f1_score, 'accuracy'])

print(model.summary())
model.fit(X_pad_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_56 (Embedding)     (None, 20, 32)            718976    
_________________________________________________________________
lstm_59 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
189616/189616 [==============================] - 99s 523us/step - loss: 0.6645 - recall: 0.6034 - precision: 0.5829 - f1_score: nan - acc: 0.5897
Epoch 2/10
189616/189616 [==============================] - 88s 466us/step - loss: 0.6384 - recall: 0.6390 - precision: 0.6283 - f1_score: 0.6273 - acc: 0.6289
Epoch 3/10
189616/189616 [==============================] - 97s 511us/step - l

In [261]:
max_review_length = 25
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

25
1


In [262]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[recall, precision,f1_score, 'accuracy'])

print(model.summary())
model.fit(X_pad_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_57 (Embedding)     (None, 25, 32)            718976    
_________________________________________________________________
lstm_60 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
189616/189616 [==============================] - 112s 588us/step - loss: 0.6637 - recall: 0.6081 - precision: 0.5867 - f1_score: nan - acc: 0.5910 - loss: 0.6638 - recall: 0.6077 - precision: 0.5865 - f1_score: nan -
Epoch 2/10
189616/189616 [==============================] - 98s 519us/step - loss: 0.6380 - recall: 0.6428 - precision: 0.6271 - f1_score: 0.6287 - acc: 0.62901s - los

In [ ]:
from keras import backend as K

def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
 
 
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
 
 
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
 
 
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
 
 
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
 
 
    return numerator / (denominator + K.epsilon())




def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

#you can use it like this
#model.compile(loss='binary_crossentropy',
#              optimizer= "adam",
#              metrics=[mcor,recall, f1])

In [243]:
max_review_length = 30
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

30
1


In [257]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=[recall, precision,f1_score, 'accuracy'])

print(model.summary())
model.fit(X_pad_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_54 (Embedding)     (None, 30, 32)            718976    
_________________________________________________________________
lstm_57 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
189616/189616 [==============================] - 125s 661us/step - loss: 0.6857 - recall: 0.5546 - precision: 0.5251 - f1_score: nan - acc: 0.5400
Epoch 2/10
189616/189616 [==============================] - 113s 598us/step - loss: 0.6644 - recall: 0.6062 - precision: 0.5947 - f1_score: 0.5908 - acc: 0.5932
Epoch 3/10
189616/189616 [==============================] - 118s 625us/step 

In [263]:
max_review_length = 40
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

40
1


In [264]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=[recall, precision,f1_score, 'accuracy'])

print(model.summary())
model.fit(X_pad_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_58 (Embedding)     (None, 40, 32)            718976    
_________________________________________________________________
lstm_61 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
189616/189616 [==============================] - 154s 814us/step - loss: 0.6877 - recall: 0.5017 - precision: 0.5150 - f1_score: nan - acc: 0.5356
Epoch 2/10
189616/189616 [==============================] - 138s 726us/step - loss: 0.6669 - recall: 0.5993 - precision: 0.5907 - f1_score: 0.5860 - acc: 0.5898
Epoch 3/10
189616/189616 [==============================] - 131s 693us/step 

In [224]:
max_review_length = 50
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

50
1


In [225]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_pad_train, y_train, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 50, 32)            718976    
_________________________________________________________________
lstm_38 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
189616/189616 [==============================] - 166s 876us/step - loss: 0.6636 - acc: 0.5910
Epoch 2/5
189616/189616 [==============================] - 156s 823us/step - loss: 0.6358 - acc: 0.6308
Epoch 3/5
189616/189616 [==============================] - 159s 840us/step - loss: 0.6176 - acc: 0.6505
Epoch 4/5
189616/189616 [==============================] - 161s 848us/step - loss: 

In [226]:
max_review_length = 60
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

60
1


In [227]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_pad_train, y_train, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_36 (Embedding)     (None, 60, 32)            718976    
_________________________________________________________________
lstm_39 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
189616/189616 [==============================] - 179s 942us/step - loss: 0.6637 - acc: 0.5890
Epoch 2/5
189616/189616 [==============================] - 175s 921us/step - loss: 0.6370 - acc: 0.6310
Epoch 3/5
189616/189616 [==============================] - 175s 920us/step - loss: 0.6182 - acc: 0.6482
Epoch 4/5
189616/189616 [==============================] - 175s 921us/step - loss: 

In [228]:
max_review_length = 100
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]

100
1


In [229]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_pad_train, y_train, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, 100, 32)           718976    
_________________________________________________________________
lstm_40 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
189616/189616 [==============================] - 272s 1ms/step - loss: 0.6657 - acc: 0.5868
Epoch 2/5
189616/189616 [==============================] - 270s 1ms/step - loss: 0.6421 - acc: 0.6229
Epoch 3/5
189616/189616 [==============================] - 269s 1ms/step - loss: 0.6228 - acc: 0.6437
Epoch 4/5
189616/189616 [==============================] - 270s 1ms/step - loss: 0.6037 -

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [30]:
words = []
for text, rating in lines:
    for word in text.split():
        if word.lower() not in stop_words:
            words.append(word.lower())

print len(words)
word_features = nltk.FreqDist(words)
most_common = word_features.most_common()[:10000]
print most_common[:10]

2005509
[('room', 22467), ('san', 16133), ('na', 15992), ('francisco', 15565), ('street', 15351), ('bedroom', 14265), ('kitchen', 13869), ('home', 13798), ('house', 12420), ('apartment', 12390)]


In [ ]:
most_common2 = word_features.most_common()
print len(most_common2)

In [92]:
plain_words = []
embedded_words = []
word_dict = {}
for each in most_common:
    plain_words.append(each[0])
    embedded_words.append(each[1])
    word_dict[each[0]] = each[1]

In [93]:
print plain_words[5]
print embedded_words[5]
print word_dict['bedroom']

bedroom
14265
14265


In [60]:
def word_embedding(descriptions):
    words = []
    for word in descriptions.split(" "):
        words.append(word)
    features = []
    for each in words:
        if each in plain_words:
            features.append(word_dict[each])
        elif each not in just_words:
            features.append(0)
    return features

In [84]:
test = lines[:10]
featuresets = [(word_embedding(descriptions), tag) for (descriptions, tag) in test]

sentences = []
feature_tags = []

for each in featuresets:
    sentences.append(each[0])
    if each[1] == 'good':
        feature_tags.append(each[1])
    elif each[1] == 'bad':
        feature_tags.append(each[0])
    
X_train = sentences[:5]
y_train = feature_tags[:5]

X_test = sentences[5:]
y_test = feature_tags[5:]

In [33]:
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [136]:
#from keras.datasets import imdb

## load the dataset but only keep the top n words, zero the rest
#top_words = 5000
#(X1_train, y1_train), (X1_test, y1_test) = imdb.load_data(num_words=top_words)
print (X1_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [121]:
print X1_train[0]
print y1_train[3000]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
0


In [96]:
#def feature_sep(descriptions):
#    words = []
##    for word in descriptions.split(" "):
 #       words.append(word)
 #   features = []
#    for each in words:
#        if each in just_words:
#            features.append(1)
#        elif each not in just_words:
##            features.append(0)
#    return features
def word_embedding(descriptions):
    words = []
    for word in descriptions.split(" "):
        words.append(word)
    features = []
    for each in words:
        if each in plain_words:
            features.append(word_dict[each])
        elif each not in just_words:
            features.append(0)
    return features

In [123]:
featuresets = [(word_embedding(descriptions), tag) for (descriptions, tag) in lines]

sentences = []
feature_tags = []

for each in featuresets:
    sentences.append(each[0])
    if each[1] == 'good':
        feature_tags.append(1)
    elif each[1] == 'bad':
        feature_tags.append(0)
    
X_train = sentences[:len(sentences)/2]
y_train = feature_tags[:len(feature_tags)/2]
X_test = sentences[len(sentences)/2:]
y_test = feature_tags[len(feature_tags)/2:]

In [137]:
# truncate and pad input sequences
max_review_length = 800
X_pad_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_pad_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print len(X_pad_train[0])
print y_train[0]


800
1


In [129]:
# create the model
embedding_vecor_length = 32
top_words = 22468
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_pad_train, y_train, validation_data=(X_pad_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1000, 32)          718976    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Train on 3502 samples, validate on 3502 samples
Epoch 1/3
3502/3502 [==============================] - 56s 16ms/step - loss: 0.2258 - acc: 0.9654 - val_loss: 0.0919 - val_acc: 0.9817
Epoch 2/3
3502/3502 [==============================] - 55s 16ms/step - loss: 0.1142 - acc: 0.9760 - val_loss: 0.0921 - val_acc: 0.9817
Epoch 3/3
3502/3502 [==============================] - 55s 16ms/step - loss: 

In [138]:
# create the model
top_words = 22468
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_pad_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_pad_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 800, 32)           718976    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 772,277
Trainable params: 772,277
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
3502/3502 [==============================] - 46s 13ms/step - loss: 0.2299 - acc: 0.9617
Epoch 2/3
3502/3502 [==============================] - 45s 13ms/step - loss: 0.1148 - acc: 0.9760
Epoch 3/3
3502/3502 [==============================] - 44s 13ms/step - loss: 0.1137 - acc: 0.9760
Accuracy: 98.17%
